In [1]:
import pandas as pd
import numpy as np
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [2]:
h2o.init()

train = h2o.import_file('./train.csv')
test = h2o.import_file('./test.csv')
sample_submission = h2o.import_file('./gender_submission.csv')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.11+9-LTS-194, mixed mode)
  Starting server from C:\Users\Gerd\venvs\h2o\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Gerd\AppData\Local\Temp\tmpfmnersr5
  JVM stdout: C:\Users\Gerd\AppData\Local\Temp\tmpfmnersr5\h2o_Gerd_started_from_python.out
  JVM stderr: C:\Users\Gerd\AppData\Local\Temp\tmpfmnersr5\h2o_Gerd_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,2 months and 18 days
H2O_cluster_name:,H2O_from_python_Gerd_fx4fun
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.975 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [18]:
train.summary()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
type,int,int,int,string,enum,real,int,int,int,real,enum,enum
mins,1.0,0.0,1.0,NaN,,0.42,0.0,0.0,693.0,0.0,,
mean,446.0,0.3838383838383838,2.3086419753086447,NaN,,29.69911764705884,0.5230078563411893,0.3815937149270483,260318.5491679275,32.20420796857465,,
maxs,891.0,1.0,3.0,NaN,,80.0,8.0,6.0,3101298.0,512.3292,,
sigma,257.3538420152301,0.4865924542648575,0.8360712409770491,NaN,,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,,
zeros,0,549,0,0,,0,608,678,0,15,,
missing,0,0,0,0,0,177,0,0,230,0,687,2
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,nan,7.25,,S
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1.0,0.0,nan,71.2833,C85,C
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,nan,7.925,,S


In [3]:
#drop the cabin column as it contains alot of missing values
#train = train.drop(['Cabin'], axis = 1)
#train = train.drop(['PassengerId'], axis = 1)

train.impute('Age', method = 'mean')

predictors = train.columns
predictors.remove('Survived')
predictors.remove('Ticket')
predictors.remove('PassengerId')

target = 'Survived'
train['Survived'] = train['Survived'].asfactor()

train = train.as_data_frame()
train['Cabin'] = train['Cabin'].str.replace('\d+', '', regex=True)
train['Cabin']
train = h2o.H2OFrame(train)


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [22]:
train.summary()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
type,int,int,int,string,enum,real,int,int,int,real,enum,enum
mins,1.0,0.0,1.0,NaN,,0.42,0.0,0.0,693.0,0.0,,
mean,446.0,0.3838383838383838,2.3086419753086447,NaN,,29.6991176470588,0.5230078563411893,0.3815937149270483,260318.5491679275,32.20420796857465,,
maxs,891.0,1.0,3.0,NaN,,80.0,8.0,6.0,3101298.0,512.3292,,
sigma,257.3538420152301,0.4865924542648575,0.8360712409770491,NaN,,13.002015226002893,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,,
zeros,0,549,0,0,,0,608,678,0,15,,
missing,0,0,0,0,0,0,0,0,230,0,0,0
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,nan,7.25,nan,S
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1.0,0.0,nan,71.2833,C,C
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,nan,7.925,nan,S


In [4]:
predictors

['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']

In [5]:
train_split, test_split, valid_split = train.split_frame([0.8, 0.1])

In [17]:
glm_model = H2OGeneralizedLinearEstimator(family= "binomial",
                                          model_id = "titanic-glm",
                                         alpha=0)

glm_model.train(predictors, target, training_frame=train_split, validation_frame=valid_split)

H2OConnectionError: Connection was closed, and can no longer be used.

In [11]:
glm_model.model_performance(test_split)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.1276838313259819
RMSE: 0.35732874405228293
LogLoss: 0.40755132360362356
Null degrees of freedom: 79
Residual degrees of freedom: 55
Null deviance: 105.85342573752904
Residual deviance: 65.20821177657977
AIC: 115.20821177657977
AUC: 0.8723333333333333
AUCPR: 0.8540941901685132
Gini: 0.7446666666666666

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3092571756538522: 


,,0,1,Error,Rate
0,0,39.0,11.0,0.22,(11.0/50.0)
1,1,4.0,26.0,0.1333,(4.0/30.0)
2,Total,43.0,37.0,0.1875,(15.0/80.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.309257,0.776119,35.0
1,max f2,0.309257,0.828025,35.0
2,max f0point5,0.729032,0.833333,14.0
3,max accuracy,0.531357,0.825000,23.0
4,max precision,0.987247,1.000000,0.0
5,max recall,0.058013,1.000000,72.0
6,max specificity,0.987247,1.000000,0.0
7,max absolute_mcc,0.309257,0.627901,35.0
8,max min_per_class_accuracy,0.335985,0.780000,33.0
9,max mean_per_class_accuracy,0.309257,0.823333,35.0



Gains/Lift Table: Avg response rate: 37,50 %, avg score: 36,45 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.0125,0.967998,2.666667,2.666667,1.000000,0.987247,1.000000,0.987247,0.033333,0.033333,166.666667,166.666667,0.033333
1,2,0.0250,0.956281,2.666667,2.666667,1.000000,0.962881,1.000000,0.975064,0.033333,0.066667,166.666667,166.666667,0.066667
2,3,0.0375,0.949304,2.666667,2.666667,1.000000,0.951501,1.000000,0.967210,0.033333,0.100000,166.666667,166.666667,0.100000
3,4,0.0500,0.942332,2.666667,2.666667,1.000000,0.945562,1.000000,0.961798,0.033333,0.133333,166.666667,166.666667,0.133333
4,5,0.0500,0.926380,0.000000,2.666667,0.000000,0.000000,1.000000,0.961798,0.000000,0.133333,-100.000000,166.666667,0.133333
5,6,0.1000,0.875748,2.666667,2.666667,1.000000,0.906033,1.000000,0.933915,0.133333,0.266667,166.666667,166.666667,0.266667
6,7,0.1500,0.781706,2.666667,2.666667,1.000000,0.844580,1.000000,0.904137,0.133333,0.400000,166.666667,166.666667,0.400000
7,8,0.2000,0.678605,2.000000,2.500000,0.750000,0.734797,0.937500,0.861802,0.100000,0.500000,100.000000,150.000000,0.480000
8,9,0.3000,0.500715,1.666667,2.222222,0.625000,0.628605,0.833333,0.784069,0.166667,0.666667,66.666667,122.222222,0.586667
9,10,0.4125,0.339949,0.888889,1.858586,0.333333,0.418667,0.696970,0.684414,0.100000,0.766667,-11.111111,85.858586,0.566667


In [12]:
predictions = glm_model.predict(test)

glm prediction progress: |████████████████████████████████████████████████| 100%


c:\users\gerd\venvs\h2o\lib\site-packages\h2o\job.py:72: UserWarning: Test/Validation dataset column 'Cabin' has levels not trained on: ["A11", "A18", "A21", "A29", "A34", "A9", "B10", "B11", "B24", "B26", ...54 not listed..., "E50", "E52", "E60", "F E46", "F E57", "F G63", "F2", "F33", "F4", "G6"]
  warnings.warn(w)


In [13]:
p = predictions.as_data_frame()
t = test.as_data_frame()

In [14]:
result = pd.concat([t, p], axis=1)
result  = result[['PassengerId','predict']]
#result['Survived'] = np.where(result['predict'] >= 0.5, 1, 0)
result['Survived'] = result['predict']
result = result.drop(['predict'], axis = 1)

In [15]:
result.to_csv('./my_submission.csv', index=False)

In [16]:
h2o.cluster().shutdown()

H2O session _sid_abda closed.
